In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

prompts = pd.read_csv('prompts_train.csv')
summaries = pd.read_csv('summaries_train.csv')

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

In [ ]:
model = AutoModel.from_pretrained(model_name)

In [54]:
fe1_df = pd.read_csv('fe_1.csv')
columns = ['text_count','average_token_length','wf_relativity', 'punct_count','wf_prompt','misspelled_words']

In [64]:
X1 = fe1_df[columns]
y1 = fe1_df[['content','wording']]


# Simple Models (No embeddings)

In [3]:
df = pd.read_csv('NOGAWANOGAWA_train.csv')

In [72]:
X = df[['summary_length','splling_err_num', 'prompt_length', 'length_ratio', 'word_overlap_count', 'bigram_overlap_count', 'bigram_overlap_ratio','trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count']]
y = df[['content','wording']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1,y1,test_size=0.2,random_state=42)

from sklearn import preprocessing
X_train_norm = preprocessing.StandardScaler().fit(X_train).transform(X_train.astype(float))
X_test_norm = preprocessing.StandardScaler().fit(X_test).transform(X_test.astype(float))

In [73]:
from sklearn.linear_model import Ridge

RigeModel=Ridge(alpha=1)

RigeModel.fit(X_train_norm, y_train)

yhat = RigeModel.predict(X_test_norm)

MCRMSE(yhat)

0.5691542502998646

In [67]:
from sklearn.metrics import mean_squared_error

def MCRMSE(yhat):
    mse1 = mean_squared_error(y_test.values[:,0],yhat[:,0])
    mse2 = mean_squared_error(y_test.values[:,1],yhat[:,1])
    return (mse1 + mse2)/2

In [75]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#Build the model
reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.01)
reg.fit(X_train_norm, y_train,
        eval_set=[(X_train_norm, y_train), (X_test_norm, y_test)],
        verbose=100)

[20:21:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.16027	validation_1-rmse:1.17788
[100]	validation_0-rmse:0.75540	validation_1-rmse:0.79021
[200]	validation_0-rmse:0.66793	validation_1-rmse:0.70908
[300]	validation_0-rmse:0.64500	validation_1-rmse:0.68869
[400]	validation_0-rmse:0.63573	validation_1-rmse:0.68256
[500]	validation_0-rmse:0.63042	validation_1-rmse:0.68067
[600]	validation_0-rmse:0.62594	validation_1-rmse:0.67968
[700]	validation_0-rmse:0.62185	validation_1-rmse:0.67917
[800]	validation_0-rmse:0.61825	validation_1-rmse:0.67914
[836]	validation_0-rmse:0.61713	validation_1-rmse:0.67913


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [77]:
MCRMSE(reg.predict(X_test_norm))

0.4610963923743292